# Part 1

## import libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Scrape table from wikipeida

In [51]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(my_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content().rstrip('\n')
    col.append((name,[]))


In [52]:
for j in range(1,len(tr_elements)):

    T=tr_elements[j]
    

    if len(T)!=3:
        break
    

    i=0
    

    for t in T.iterchildren():
        data=t.text_content().rstrip('\n') 

        if i>0:

            try:
                data=int(data)
            except:
                pass

        col[i][1].append(data)

        i+=1

In [89]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [90]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## remove NA from Borough

In [91]:
df=df[df.Borough!='Not assigned'].reset_index(drop=True)

In [92]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [100]:
df.shape

(104, 3)